## Split sarcasn on reddit dataset for the purposes of the project
This file will split the roughly 1M entries from the kaggle dataset into
100K Holdout for testing and eval
300K For SFT training and validation
300K For RM training and validataion
300K for PPO training

In [1]:
import datasets
from datasets import load_dataset, DatasetDict

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
!git clone https://huggingface.co/datasets/marcbishara/sarcasm-on-reddit

Cloning into 'sarcasm-on-reddit'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 2), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (22/22), 6.13 KiB | 1.53 MiB/s, done.
Filtering content: 100% (8/8), 417.54 MiB | 25.02 MiB/s, done.


In [11]:
dataset = load_dataset("csv", data_files="https://huggingface.co/datasets/marcbishara/sarcasm-on-reddit/resolve/main/train-balanced-sarcasm.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
dataset = dataset['train']

In [19]:
dataset = dataset.shuffle(seed=42)

In [20]:
dataset = dataset.cast_column("label", datasets.ClassLabel(names=["not_sarcastic", "sarcastic"]))

Casting the dataset:   0%|          | 0/1010826 [00:00<?, ? examples/s]

In [21]:
print(dataset.features["label"].int2str(1))

sarcastic


In [22]:
split = dataset.train_test_split(test_size=0.1, stratify_by_column="label", seed=42)

In [23]:
holdout = split["test"]
remaining = split["train"]

In [24]:
num_label1 = holdout.filter(lambda x: x["label"] == 1).num_rows
print("Rows with label=1 in holdout:", num_label1)

Filter:   0%|          | 0/101083 [00:00<?, ? examples/s]

Rows with label=1 in holdout: 50542


In [25]:
print("Total rows in remaining dataset:", len(remaining))

# Split remaining into 3 equal parts
n = len(remaining)
third = n // 3
sft = remaining.select(range(0, third))
reward = remaining.select(range(third, 2*third))
ppo = remaining.select(range(2*third, n))

Total rows in remaining dataset: 909743


In [26]:
# Train/Validate splits (90/10) for each
def train_val_split(ds, seed=42):
    ds = ds.shuffle(seed=seed)
    n = len(ds)
    train_n = int(n * 0.9)
    train = ds.select(range(train_n))
    val = ds.select(range(train_n, n))
    return DatasetDict({"train": train, "validation": val})

In [27]:
sft = train_val_split(sft)
reward = train_val_split(reward)
ppo = train_val_split(ppo)

In [28]:
# Final DatasetDict
dataset_dict = DatasetDict({
    "holdout": holdout,
    "sft_train": sft["train"],
    "sft_validation": sft["validation"],
    "reward_train": reward["train"],
    "reward_validation": reward["validation"],
    "ppo_train": ppo["train"],
    "ppo_validation": ppo["validation"],
})

In [29]:
dataset_dict

DatasetDict({
    holdout: Dataset({
        features: ['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'],
        num_rows: 101083
    })
    sft_train: Dataset({
        features: ['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'],
        num_rows: 272922
    })
    sft_validation: Dataset({
        features: ['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'],
        num_rows: 30325
    })
    reward_train: Dataset({
        features: ['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'],
        num_rows: 272922
    })
    reward_validation: Dataset({
        features: ['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'],
        num_rows: 30325
    })
    ppo_train: Dataset({
        features: [

In [30]:
holdout_lbl_cnt = dataset_dict['holdout'].filter(lambda x: x["label"] == 1).num_rows
sft_train_lbl_cnt = dataset_dict['sft_train'].filter(lambda x: x["label"] == 1).num_rows
sft_val_lbl_cnt = dataset_dict['sft_validation'].filter(lambda x: x["label"] == 1).num_rows
reward_train_lbl_cnt = dataset_dict['reward_train']. filter(lambda x: x["label"] == 1).num_rows
reward_val_lbl_cnt = dataset_dict['reward_validation'].filter(lambda x: x["label"] == 1).num_rows
ppo_train_lbl_cnt = dataset_dict['ppo_train'].filter(lambda x: x["label"] == 1).num_rows
ppo_val_lbl_cnt = dataset_dict['ppo_validation'].filter(lambda x: x["label"] == 1).num_rows

Filter:   0%|          | 0/101083 [00:00<?, ? examples/s]

Filter:   0%|          | 0/272922 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30325 [00:00<?, ? examples/s]

Filter:   0%|          | 0/272922 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30325 [00:00<?, ? examples/s]

Filter:   0%|          | 0/272924 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30325 [00:00<?, ? examples/s]

In [31]:
print(f"Holdout label balance is {round(holdout_lbl_cnt / len(dataset_dict['holdout']) * 100, 2)}% and has {len(dataset_dict['holdout'])} items")
print(f"SFT train label balance is {round(sft_train_lbl_cnt / len(dataset_dict['sft_train']) * 100, 2)}% and has {len(dataset_dict['sft_train'])} items")
print(f"SFT val label balance is {round(sft_val_lbl_cnt / len(dataset_dict['sft_validation']) * 100, 2)}% and has {len(dataset_dict['sft_validation'])} items")
print(f"Reward train label balance is {round(reward_train_lbl_cnt / len(dataset_dict['reward_train']) * 100, 2)}% and has {len(dataset_dict['reward_train'])} items")
print(f"Reward val label balance is {round(reward_val_lbl_cnt / len(dataset_dict['reward_validation']) * 100, 2)}% and has {len(dataset_dict['reward_validation'])} items")
print(f"PPO train label balance is {round(ppo_train_lbl_cnt / len(dataset_dict['ppo_train']) * 100, 2)}% and has {len(dataset_dict['ppo_train'])} items")
print(f"PPO val label balance is {round(ppo_val_lbl_cnt / len(dataset_dict['ppo_validation']) * 100, 2)}% and has {len(dataset_dict['ppo_validation'])} items")

Holdout label balance is 50.0% and has 101083 items
SFT train label balance is 50.07% and has 272922 items
SFT val label balance is 49.75% and has 30325 items
Reward train label balance is 49.99% and has 272922 items
Reward val label balance is 50.41% and has 30325 items
PPO train label balance is 49.98% and has 272924 items
PPO val label balance is 49.51% and has 30325 items


In [32]:
dataset_dict.push_to_hub("marcbishara/sarcasm-on-reddit")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/102 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 18.2MB / 18.2MB            

...

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 5.51MB / 5.51MB            

README.md: 0.00B [00:00, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/marcbishara/sarcasm-on-reddit/commit/c6a5fdda0b6bfda983619ce333d6ddbb3de11a71', commit_message='Upload dataset', commit_description='', oid='c6a5fdda0b6bfda983619ce333d6ddbb3de11a71', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/marcbishara/sarcasm-on-reddit', endpoint='https://huggingface.co', repo_type='dataset', repo_id='marcbishara/sarcasm-on-reddit'), pr_revision=None, pr_num=None)